In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [20]:
ticker = sorted(['AAPL', 'MSFT', 'GOOGL'])
data = yf.download(ticker, start='2020-01-01', end='2025-01-01')['Close']
data.to_excel('data.xlsx')

C:\Users\samue\AppData\Local\Temp\ipykernel_11496\3779400190.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start='2020-01-01', end='2025-01-01')['Close']
[*********************100%***********************]  3 of 3 completed


In [21]:
rets.cov()

Ticker,AAPL,GOOGL,MSFT
Ticker,,,
AAPL,0.000398,0.000265,0.000287
GOOGL,0.000265,0.000419,0.000293
MSFT,0.000287,0.000293,0.000369


In [22]:
def opt_min_var(tickers: list, start: str, end: str) -> np.array:
    # Obtener precios
    prices = yf.download(tickers, start=start, end=end, progress=False)['Close']

    # Obtener rendimiento
    returns = prices.pct_change().dropna()

    # Obtener rendimiento promedio
    mean_returns = returns.mean()

    # Obtener matriz de covariânza varianza
    cov = returns.cov()

    # Definir funcion objetivo
    varianza = lambda w: w.T @ cov @ w

    # Definir valores iniciales
    n = len(tickers)
    w_inicial = np.ones(n) / n

    # Definir las cotas
    cotas = [(0, 1)] * n

    # Definir las restricciones --> Debe estar igualada a 0 en scipy
    restricciones = lambda w: np.sum(w) - 1

    # Definir tolerancia
    tol = 1e-20

    # Aplicar función de scipy.optimize
    result = minimize(fun = varianza, 
             x0 = w_inicial, 
             bounds = cotas, 
             constraints = {'fun': restricciones, 'type': 'eq'}, 
             tol = tol)

    # Encontrar pesos eficientes 
    w_opt = result.x

    # Rendimiento del potafolio
    rend_p = np.dot(w_opt, mean_returns)

    # Volatilidad del portafolio
    vol_p = np.sqrt(w_opt.T @ cov @ w_opt)

    return w_opt, rend_p, vol_p

In [23]:
w, rend, vol = opt_min_var(ticker, '2020-01-01', '2025-01-01')
w

C:\Users\samue\AppData\Local\Temp\ipykernel_11496\3034305429.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start=start, end=end, progress=False)['Close']


array([0.3459348 , 0.28315202, 0.37091318])

In [24]:
vol**2

np.float64(0.0003192404488941707)